# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dlnco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dlnco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dlnco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.multioutput import MultiOutputClassifier


from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///categorized_messages.db')
df = pd.read_sql('select * from categorized_messages',engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y_cols = ['related', 'request', 'offer',
          'aid_related', 'medical_help', 'medical_products', 
          'search_and_rescue', 'security', 'military', 
          'child_alone', 'water', 'food', 'shelter',
          'clothing', 'money', 'missing_people', 'refugees', 
          'death', 'other_aid', 'infrastructure_related', 
          'transport', 'buildings', 'electricity', 'tools', 
          'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
          'weather_related', 'floods', 'storm', 'fire', 
          'earthquake', 'cold', 'other_weather', 'direct_report']

In [5]:
X = df['message']
Y = df[Y_cols]

### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

Pipeline: <br>
---Bag of Words <br>
---TFIDF <br>
---MultiOutput<br>
----Gaussian Naive-Bayes

In [7]:
# class used to transform data to an acceptable format for GaussianNB
class DenseTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('toarray', DenseTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000226C391D790>)),
                ('tfidf', TfidfTransformer()), ('toarray', DenseTransformer()),
                ('clf', MultiOutputClassifier(estimator=GaussianNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

# transform to dataframe to be easier to read
df_pred = pd.DataFrame(y_pred)
df_pred.columns = Y_cols

for col in df_pred.columns:
    print('Analysed class: '+ str(col) + '\n')
    print(classification_report(y_test[col], 
                                df_pred[col]))

Analysed class: related

              precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.86      1.00      0.93        25

    accuracy                           0.87        30
   macro avg       0.93      0.60      0.63        30
weighted avg       0.89      0.87      0.83        30

Analysed class: request

              precision    recall  f1-score   support

           0       0.90      0.64      0.75        14
           1       0.75      0.94      0.83        16

    accuracy                           0.80        30
   macro avg       0.82      0.79      0.79        30
weighted avg       0.82      0.80      0.79        30

Analysed class: offer

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        29
           1       0.00      0.00      0.00         1

    accuracy                           0.97        30
   macro avg       0.48      0.50      0.49        30
w

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# try bernoulli for short texts.
pipeline_bernoulli = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('toarray', DenseTransformer()),
    ('clf', MultiOutputClassifier(BernoulliNB()))
])

pipeline_bernoulli.fit(X_train, y_train)

y_pred_bernoulli = pipeline_bernoulli.predict(X_test)

# transform to dataframe to be easier to read
df_pred_bernoulli = pd.DataFrame(y_pred_bernoulli)
df_pred_bernoulli.columns = Y_cols

for col in df_pred.columns:
    print('Analysed class: '+ str(col) + '\n')
    print('Naive Bayes classification: \n')
    print(classification_report(y_test[col], 
                            df_pred[col]))
    print('Bernoulli Naive Bayes classification: \n')
    print(classification_report(y_test[col], 
                                df_pred_bernoulli[col]))

Analysed class: related

Naive Bayes classification: 

              precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.86      1.00      0.93        25

    accuracy                           0.87        30
   macro avg       0.93      0.60      0.63        30
weighted avg       0.89      0.87      0.83        30

Bernoulli Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.83      1.00      0.91        25

    accuracy                           0.83        30
   macro avg       0.42      0.50      0.45        30
weighted avg       0.69      0.83      0.76        30

Analysed class: request

Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.90      0.64      0.75        14
           1       0.75      0.94      0.83        16

    accuracy                    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

Bernoulli Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

Analysed class: electricity

Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        29
           1       0.00      0.00      0.00         1

    accuracy                           0.97        30
   macro avg       0.48      

In [11]:
parameters = {
    'clf__estimator__alpha': (0.1, 0.5, 0.9)
}

cv = GridSearchCV(pipeline_bernoulli, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)

# transform to dataframe to be easier to read
df_pred_cv = pd.DataFrame(y_pred_cv)
df_pred_cv.columns = Y_cols


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
for col in df_pred.columns:
    print('Analysed class: '+ str(col) + '\n')
    print('Bernoulli Naive Bayes classification: \n')
    print(classification_report(y_test[col], 
                                df_pred_bernoulli[col]))
    print('Bernoulli Naive Bayes (with GridSearch) classification: \n')
    print(classification_report(y_test[col], 
                                df_pred_cv[col]))

Analysed class: related

Bernoulli Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.83      1.00      0.91        25

    accuracy                           0.83        30
   macro avg       0.42      0.50      0.45        30
weighted avg       0.69      0.83      0.76        30

Bernoulli Naive Bayes (with GridSearch) classification: 

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.88      0.92      0.90        25

    accuracy                           0.83        30
   macro avg       0.69      0.66      0.67        30
weighted avg       0.82      0.83      0.83        30

Analysed class: request

Bernoulli Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.75      0.43      0.55        14
           1       0.64      0.88      0.74      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Bernoulli Naive Bayes (with GridSearch) classification: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

Analysed class: buildings

Bernoulli Naive Bayes classification: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

Bernoulli Naive Bayes (with GridSearch) classification: 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [13]:
import pickle 
pickle.dump(pipeline, open('GaussianNB.sav','wb'))
pickle.dump(pipeline_bernoulli, open('BernoulliNB.sav','wb'))
pickle.dump(cv, open('BernoulliNB_withGridSearch.sav','wb'))


In [ ]:
test = pickle.load(open('GaussianNB.sav','rb'))
test.predict(X_test)

array([[1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1]])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.